In [8]:
!pip uninstall pyarrow -q
!pip install --upgrade pyarrow -q
!pip install transformers -q
!pip install datasets -q
!pip install evaluate -q

Proceed (Y/n)? Y
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pyarrow<18.0.0a0,>=14.0.0, but you have pyarrow 18.0.0 which is incompatible.
pylibcudf-cu12 24.10.1 requires pyarrow<18.0.0a0,>=14.0.0, but you have pyarrow 18.0.0 which is incompatible.


In [1]:
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoModelForCausalLM, AutoModelForMaskedLM
from transformers import BertModel, BertConfig, BertTokenizer
from transformers import AdamW
from transformers import get_scheduler
from transformers import DataCollatorWithPadding, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import evaluate

import wandb
wandb.init(mode='disabled')

import torch
from torch.utils.data import DataLoader

import numpy as np
import math

from tqdm.auto import tqdm
import html
import requests

In [2]:
checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [3]:
text = "The election was lost due to [MASK]."

In [4]:
model_inputs = tokenizer(text, return_tensors='pt')
outputs = model(**model_inputs).logits
mask_token_index = torch.where(model_inputs["input_ids"] == tokenizer.mask_token_id)[1]
top_5_tokens = torch.topk(outputs[0, mask_token_index, :], 5, dim=1).indices[0].tolist()
tokenizer.decode(top_5_tokens)

'irregularities fraud bribery redistribution recounts'

In [5]:
tokenizer.mask_token_id

103

In [6]:
tokens = tokenizer.tokenize(text)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(tokens)
print(token_ids)

['the', 'election', 'was', 'lost', 'due', 'to', '[MASK]', '.']
[1996, 2602, 2001, 2439, 2349, 2000, 103, 1012]


In [7]:
print(outputs.shape)

torch.Size([1, 10, 30522])


In [8]:
imdb_dataset = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [10]:
imdb_dataset["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [11]:
sample = imdb_dataset["unsupervised"].shuffle().select(range(3))

In [12]:
for s in sample:
  print(s['text'])
  print(s['label'])

"Ballad of a Soldier" along with "Andrei Rublev" is one of my favorite Russian films ever and one of the great movies made in USSR during Khruschev´s so-called "thaw" period in the late 1950s and early 1960s when soviet filmmakers got a certain amount of an artistic freedom and were less controlled by the Soviet state.<br /><br />Directed and co-written by master Russian filmmaker of a soviet era, and also a WW-2 veteran, Grigory Chukhrai, who was even nominated for this picture for Academy Award for Best Screenplay, the film is set during WW-2. It´s a story of 19 year old Russian soldier who as a reward for a heroic act in fighting with Germans given a 3 day leave home by his commanding officer. We follow the soldier´s journey home through ravished by war land where he meets different people and witnesses how they lives are affected by war and falls in love with a girl whom he occasionally meets on a train. The atmosphere created by this movie is quite unique, rarely before or after a

In [17]:
# context_length = 128
# def tokenize_and_chunk(texts):
#   all_input_ids = []
#   for input_ids in tokenizer(texts['text'])['input_ids']:
#     all_input_ids.extend(input_ids)
#     all_input_ids.append(tokenizer.eos_token_id)

#   chunks = []
#   for i in range(0, len(all_input_ids), context_length):
#     chunks.append(all_input_ids[i: i+context_length])

#   return {'input_ids': chunks}

# tokenized_dataset = imdb_dataset.map(tokenize_and_chunk, batched=True, remove_columns=['text'])

In [18]:
def tokenize_function(examples):
  result = tokenizer(examples["text"])
  result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
  return result

In [19]:
tokenized_datasets = imdb_dataset.map(tokenize_function, batched=True, remove_columns=["text", "label"])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [20]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [21]:
for k in tokenized_datasets["train"][0]:
  print(k, tokenized_datasets["train"][0][k])

input_ids [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 2055, 2037, 10740, 2006, 4331, 1010, 2016, 2038, 3348, 2007, 2014, 3689, 3836, 1

In [22]:
chunk_size = 128

In [23]:
tokenized_samples = tokenized_datasets["train"][:3]

In [24]:
for idx, sample in enumerate(tokenized_samples['input_ids']):
  print(idx, len(sample))

0 363
1 304
2 133


In [25]:
concatenated_examples = {k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()}

In [26]:
total_length = len(concatenated_examples['input_ids'])

In [27]:
chunks = {k: [t[i: i+chunk_size] for i in range(0, total_length, chunk_size)] for k,t in concatenated_examples.items()}

In [28]:
print(len(chunks['input_ids']))

7


In [29]:
def group_texts(examples):
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  total_length = len(concatenated_examples['input_ids'])
  total_length = (total_length // chunk_size) * chunk_size

  result = {k: [t[i: i+chunk_size] for i in range(0, total_length, chunk_size)] for k,t in concatenated_examples.items()}
  result["labels"] = result["input_ids"].copy()
  return result

In [30]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [31]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

In [32]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [33]:
samples = [lm_datasets["train"][i] for i in range(3)]

for sample in samples:
  _ = sample.pop('word_ids')

for chunk in data_collator(samples)['input_ids']:
  print(tokenizer.decode(chunk))

[CLS] i rented i am [MASK] - yellow from my video store [MASK] of all the controversy that surrounded it [MASK] it was first released in 1967. [MASK] also heard that at [MASK] it was seized by u. s. customs if it ever tried to enter this country, therefore being a fan of films considered [MASK] controversial " i really had to [MASK] this for myself. < br [MASK] > < br / > the plot is centered around a young swedish dramatal [MASK] [MASK] [MASK] [MASK] to learn everything she can about life. in particular she [MASK] to focus her attentions to making some sort of documentary on what the average swede thought [MASK] certain political issues [MASK]
as the vietnam war and race issues in plotting united states. in between asking politicians and ordinary den [MASK]ns of stockholm about their opinions on politics [MASK] she [MASK] sex with her drama teacher, classmates [MASK] and married men. < br / > < br / > what kills me about i am curious - [MASK] is that 40 [MASK] ago, this was considered

In [34]:
train_size = 10000
test_size = int(0.1 * train_size)

In [36]:
downsampled_dataset = lm_datasets["train"].train_test_split(train_size=train_size, test_size=test_size, seed=42)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [38]:
batch_size = 64
logging_steps = len(downsampleld_dataset["train"]) // batch_size
model_name = checkpoint.split("/")[-1]

In [39]:
training_args = TrainingArguments(
    output_dir = f"{model_name}-finetuned_imdb",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    logging_steps=logging_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

<ipython-input-41-cea8c98cdfa5>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss


In [ ]:
def insert_random_mask(batch):
  features = [dict(zip(batch, t)) for t in zip(*batch.values())]
  masked_inputs = data_collator(features)
  return {"masked_"+k: v.numpy() for k,v in masked_inputs.items()}

In [ ]:
downsampled_dataset = downsampled_dataset.remove_columns(["word_ids"])
eval_dataset = downsampled_dataset["test"].map(insert_random_mask, batched=True, remove_columns=downsampled_dataset["test"].column_names)
eval_dataset = eval_dataset.rename_columns({"masked_input_ids": "input_ids", "masked_attention_mask": "attention_mask", "masked_labels": "labels"})

In [ ]:
batch_size = 64
train_dataloader = DataLoader(
    downsampled_dataset["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator
)
eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_dat_collator
)

In [ ]:
model = AutoModelForMaskedLM.from_pretrained(checkpoint)

In [ ]:
opt = torch.optim.AdamW(mode.paarameters(), lr=5e-5)

In [ ]:
for epoch in range(3):
  model.train()
  for batch in train_dataloader:
    outputs = model(**batch)
    loss = output.loss
    loss.backward()

    opt.step()
    opt.zero_grad()

  model.eval()
  losses = []
  for step, batch in enumerate(eval_dataloader):
    with torch.inference_mode():
      outputs = model(**batch)

    loss = outputs.loss
    losses.append(loss.detach().cpu().item())

  losses = torch.cat(losses)
  losses = losses[:len(eval_dataset)]

  perplexity = math.exp(torch.mean(losses))


In [12]:
def tokenize_function(examples):
  result = tokenizer(examples["text"])
  result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
  return result

In [13]:
tokenized_datasets = imdb_dataset.map(tokenize_function, batched=True, remove_columns=["text", "label"])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [17]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [16]:
chunk_size = 128

In [19]:
tokenized_samples = tokenized_datasets["train"][:3]

In [20]:
for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 363'
'>>> Review 1 length: 304'
'>>> Review 2 length: 133'


dict